# PDF RAG System - Interactive Demo

This notebook demonstrates the PDF RAG System capabilities including:
- Document upload and processing
- Query processing with citations
- Performance monitoring
- System management

## Prerequisites

1. Start the API server: `python run_api.py`
2. Ensure you have a PDF document to test with
3. Install required packages: `pip install requests matplotlib pandas`

In [ ]:
import requests
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display, HTML, Markdown

# Configuration
API_BASE_URL = "http://localhost:8000"
TEST_PDF_PATH = "../data/Owners_Manual.pdf"  # Adjust path as needed

print("PDF RAG System Demo - Setup Complete")

## 1. System Health Check

First, let's verify the system is running and healthy.

In [ ]:
def check_system_health():
    """Check if the API server is running and healthy."""
    try:
        response = requests.get(f"{API_BASE_URL}/health", timeout=10)
        response.raise_for_status()
        health_data = response.json()
        
        print("✅ System Health Check")
        print(f"   Status: {health_data['status']}")
        print(f"   Version: {health_data['version']}")
        print(f"   Uptime: {health_data['uptime_seconds']:.1f} seconds")
        print(f"   Memory Usage: {health_data['memory_usage_mb']:.1f} MB")
        print(f"   Total Chunks: {health_data['total_chunks']}")
        print(f"   Active Requests: {health_data['active_requests']}")
        
        return True
        
    except requests.exceptions.RequestException as e:
        print(f"❌ System Health Check Failed: {e}")
        print("   Make sure the API server is running: python run_api.py")
        return False

system_healthy = check_system_health()

## 2. Document Upload and Processing

Upload a PDF document and process it for querying.

In [ ]:
def upload_document(file_path):
    """Upload and process a PDF document."""
    if not Path(file_path).exists():
        print(f"❌ File not found: {file_path}")
        return None
    
    try:
        with open(file_path, 'rb') as f:
            files = {'file': (Path(file_path).name, f, 'application/pdf')}
            
            print(f"📄 Uploading document: {Path(file_path).name}")
            start_time = time.time()
            
            response = requests.post(f"{API_BASE_URL}/upload", files=files, timeout=60)
            response.raise_for_status()
            
            upload_time = time.time() - start_time
            result = response.json()
            
            print("✅ Document Upload Successful")
            print(f"   Filename: {result['filename']}")
            print(f"   File Size: {result['file_size']:,} bytes")
            print(f"   Pages Processed: {result['pages_processed']}")
            print(f"   Chunks Created: {result['chunks_created']}")
            print(f"   Processing Time: {result['processing_time_ms']:.1f} ms")
            print(f"   Total Upload Time: {upload_time:.1f} seconds")
            
            return result
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Document Upload Failed: {e}")
        return None

if system_healthy:
    upload_result = upload_document(TEST_PDF_PATH)
else:
    print("⏭️ Skipping upload - system not healthy")

## 3. Query Processing with Citations

Ask questions about the uploaded document and get answers with source citations.

In [ ]:
def process_query(query, include_citations=True):
    """Process a query and return the answer with citations."""
    try:
        payload = {
            "query": query,
            "include_citations": include_citations,
            "check_sufficiency": True
        }
        
        print(f"🤔 Processing Query: {query}")
        start_time = time.time()
        
        response = requests.post(f"{API_BASE_URL}/query", json=payload, timeout=30)
        response.raise_for_status()
        
        query_time = time.time() - start_time
        result = response.json()
        
        print("✅ Query Processing Successful")
        print(f"   Processing Time: {result['processing_time_ms']:.1f} ms")
        print(f"   Total Query Time: {query_time:.1f} seconds")
        print(f"   Confidence: {result['confidence']}")
        print(f"   Sufficient Information: {result['sufficient_information']}")
        print(f"   Source Count: {result['source_count']}")
        print(f"   Citations: {len(result['citations'])}")
        
        return result
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Query Processing Failed: {e}")
        return None

def display_answer(result):
    """Display the answer and citations in a formatted way."""
    if not result:
        return
    
    # Display answer
    display(Markdown(f"### Answer\n{result['answer']}"))
    
    # Display citations if available
    if result['citations']:
        display(Markdown("### Citations"))
        for i, citation in enumerate(result['citations'], 1):
            display(Markdown(f"**{i}.** {citation.get('text', 'N/A')[:200]}..."))
            if 'page' in citation:
                display(Markdown(f"   *Source: Page {citation['page']}*"))
    
    # Display metadata
    display(Markdown(f"**Confidence:** {result['confidence']} | **Processing Time:** {result['processing_time_ms']:.1f}ms"))

# Sample queries
sample_queries = [
    "What are the main safety features mentioned in this document?",
    "How do I charge or fuel this vehicle?",
    "What warranty information is provided?"
]

if system_healthy and upload_result:
    print("\n" + "="*60)
    print("QUERY PROCESSING DEMONSTRATION")
    print("="*60)
    
    query_results = []
    
    for i, query in enumerate(sample_queries, 1):
        print(f"\n--- Query {i} ---")
        result = process_query(query)
        if result:
            query_results.append(result)
            display_answer(result)
        print("\n" + "-"*40)
else:
    print("⏭️ Skipping queries - no document uploaded")
    query_results = []

## 4. Performance Analysis

Analyze query performance and system metrics.

In [ ]:
def get_system_metrics():
    """Get comprehensive system metrics."""
    try:
        response = requests.get(f"{API_BASE_URL}/metrics", timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Failed to get metrics: {e}")
        return None

def analyze_performance(query_results):
    """Analyze query performance."""
    if not query_results:
        print("No query results to analyze")
        return
    
    # Extract performance data
    processing_times = [r['processing_time_ms'] for r in query_results]
    confidences = [r['confidence'] for r in query_results]
    citation_counts = [len(r['citations']) for r in query_results]
    
    # Create performance DataFrame
    df = pd.DataFrame({
        'Query': [f"Query {i+1}" for i in range(len(query_results))],
        'Processing Time (ms)': processing_times,
        'Confidence': confidences,
        'Citations': citation_counts
    })
    
    print("📊 Query Performance Analysis")
    display(df)
    
    # Performance statistics
    print(f"\n📈 Performance Statistics:")
    print(f"   Average Processing Time: {sum(processing_times)/len(processing_times):.1f} ms")
    print(f"   Min Processing Time: {min(processing_times):.1f} ms")
    print(f"   Max Processing Time: {max(processing_times):.1f} ms")
    print(f"   Average Citations: {sum(citation_counts)/len(citation_counts):.1f}")
    
    # Confidence distribution
    confidence_dist = pd.Series(confidences).value_counts()
    print(f"\n🎯 Confidence Distribution:")
    for conf, count in confidence_dist.items():
        print(f"   {conf}: {count} queries")
    
    # Create visualizations
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Processing time chart
    axes[0].bar(df['Query'], df['Processing Time (ms)'], color='skyblue')
    axes[0].set_title('Query Processing Times')
    axes[0].set_ylabel('Time (ms)')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Citations chart
    axes[1].bar(df['Query'], df['Citations'], color='lightgreen')
    axes[1].set_title('Citations per Query')
    axes[1].set_ylabel('Number of Citations')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

if system_healthy:
    # Get system metrics
    metrics = get_system_metrics()
    if metrics:
        print("🔧 System Metrics:")
        print(f"   Uptime: {metrics['system']['uptime_seconds']:.1f} seconds")
        print(f"   Memory Usage: {metrics['system']['memory_usage_mb']:.1f} MB")
        print(f"   CPU Usage: {metrics['system']['cpu_percent']:.1f}%")
        print(f"   Total Requests: {metrics['system']['total_requests']}")
        print(f"   Active Requests: {metrics['system']['active_requests']}")
        print(f"   Total Chunks: {metrics['workflow']['total_chunks']}")
        print(f"   Caching Enabled: {metrics['workflow']['enable_caching']}")
    
    # Analyze query performance
    if query_results:
        analyze_performance(query_results)
else:
    print("⏭️ Skipping performance analysis - system not healthy")

## 5. Interactive Query Interface

Try your own queries interactively.

In [ ]:
def interactive_query():
    """Interactive query interface."""
    if not system_healthy:
        print("System not healthy - cannot process queries")
        return
    
    print("🤖 Interactive Query Interface")
    print("Enter your questions below. Type 'quit' to exit.\n")
    
    while True:
        try:
            query = input("Your question: ").strip()
            
            if query.lower() in ['quit', 'exit', 'q']:
                print("Goodbye!")
                break
            
            if not query:
                print("Please enter a question.")
                continue
            
            result = process_query(query)
            if result:
                print("\n" + "="*50)
                display_answer(result)
                print("="*50 + "\n")
            
        except KeyboardInterrupt:
            print("\nGoodbye!")
            break
        except Exception as e:
            print(f"Error: {e}")

# Uncomment the line below to start interactive mode
# interactive_query()

print("💡 Tip: Uncomment the line above to start interactive query mode")

## 6. System Management

Demonstrate cache and document management features.

In [ ]:
def test_cache_performance():
    """Test cache performance by running the same query twice."""
    if not system_healthy:
        return
    
    test_query = "What is this document about?"
    
    print("🚀 Cache Performance Test")
    
    # First query (cache miss)
    print("\n1. First query (cache miss):")
    start_time = time.time()
    result1 = process_query(test_query)
    time1 = time.time() - start_time
    
    # Second query (cache hit)
    print("\n2. Second query (cache hit):")
    start_time = time.time()
    result2 = process_query(test_query)
    time2 = time.time() - start_time
    
    if result1 and result2:
        speedup = time1 / time2 if time2 > 0 else 0
        print(f"\n📈 Cache Performance Results:")
        print(f"   First query time: {time1:.2f} seconds")
        print(f"   Second query time: {time2:.2f} seconds")
        print(f"   Speedup: {speedup:.1f}x faster")

def clear_system_cache():
    """Clear the system cache."""
    try:
        response = requests.delete(f"{API_BASE_URL}/cache", timeout=10)
        response.raise_for_status()
        result = response.json()
        print(f"✅ Cache cleared: {result['message']}")
        return True
    except requests.exceptions.RequestException as e:
        print(f"❌ Failed to clear cache: {e}")
        return False

def clear_documents():
    """Clear all documents from the system."""
    try:
        response = requests.delete(f"{API_BASE_URL}/documents", timeout=10)
        response.raise_for_status()
        result = response.json()
        print(f"✅ Documents cleared: {result['message']}")
        return True
    except requests.exceptions.RequestException as e:
        print(f"❌ Failed to clear documents: {e}")
        return False

if system_healthy and upload_result:
    # Test cache performance
    test_cache_performance()
    
    print("\n" + "="*50)
    print("SYSTEM MANAGEMENT")
    print("="*50)
    
    # Clear cache
    print("\n🧹 Clearing system cache...")
    clear_system_cache()
    
    # Note: Uncomment below to clear documents (will require re-upload)
    # print("\n🗑️ Clearing all documents...")
    # clear_documents()
    
    print("\n💡 Tip: Uncomment the lines above to clear documents (requires re-upload)")
else:
    print("⏭️ Skipping system management - no document uploaded")

## 7. Summary and Next Steps

Summary of the demonstration and suggestions for further exploration.

In [ ]:
print("🎉 PDF RAG System Demo Complete!")
print("\n📋 What we demonstrated:")
print("   ✅ System health monitoring")
print("   ✅ Document upload and processing")
print("   ✅ Query processing with citations")
print("   ✅ Performance analysis and metrics")
print("   ✅ Cache performance testing")
print("   ✅ System management operations")

print("\n🚀 Next Steps:")
print("   1. Try the interactive query interface above")
print("   2. Upload your own PDF documents")
print("   3. Experiment with different types of questions")
print("   4. Monitor system performance with /metrics endpoints")
print("   5. Integrate the API into your own applications")

print("\n📚 API Documentation:")
print(f"   Interactive docs: {API_BASE_URL}/docs")
print(f"   ReDoc: {API_BASE_URL}/redoc")
print(f"   Health check: {API_BASE_URL}/health")
print(f"   Metrics: {API_BASE_URL}/metrics")

print("\n🔧 Configuration:")
print("   - Modify src/config.py for custom settings")
print("   - Adjust chunk sizes, retrieval parameters")
print("   - Configure caching and performance options")

print("\n💡 Tips for Production:")
print("   - Monitor system metrics regularly")
print("   - Implement proper error handling")
print("   - Use appropriate timeout values")
print("   - Consider rate limiting for high-traffic scenarios")
print("   - Set up logging and monitoring")

display(HTML(f'<h3><a href="{API_BASE_URL}/docs" target="_blank">🌐 Open API Documentation</a></h3>'))